In [1]:
import numpy as np
import scipy.stats as scs
import matplotlib.pyplot as plt
import time

import kkl.divergences as dv
import kkl.kernels as kl
import kkl.gradient_descent as gd
import kkl.generate_y as gy
import cProfile